In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=44551c9c54a711543122b868f965e5d506d05c890717ec2ca27598263b71f21b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import os
import pandas as pd
import random
import zipfile

In [ ]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip


Saved under test_data.zip


In [ ]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1


Saved under products_leaflets_256.zip


In [ ]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'

# Bildgröße und Anzahl der Klassen
image_size = (128, 128)
num_classes = len(os.listdir(train_directory))

# Liste für Bilder und Labels
images = []
labels = []

# Durchlaufe das Trainingsverzeichnis und lade die Bilder
for root, dirs, files in os.walk(train_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            images.append(image_array)
            labels.append(int(os.path.basename(root)))

# Konvertiere die Listen in Numpy-Arrays
images = np.array(images)
labels = np.array(labels)

# Aufteilung der Daten in Trainings- und Validierungssets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Erstelle das CNN-Modell mit 3 Layers
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Kompilieren des Modells
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Trainieren des Modells mit 10 durchläufen
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
832/832 [==============================] - 741s 889ms/step - loss: 9.8390 - accuracy: 0.0090 - val_loss: 6.2246 - val_accuracy: 0.0278
Epoch 2/10
832/832 [==============================] - 737s 886ms/step - loss: 5.8232 - accuracy: 0.0645 - val_loss: 5.8314 - val_accuracy: 0.0741
Epoch 3/10
832/832 [==============================] - 730s 878ms/step - loss: 4.7410 - accuracy: 0.1771 - val_loss: 5.6889 - val_accuracy: 0.1143
Epoch 4/10
832/832 [==============================] - 690s 830ms/step - loss: 3.4981 - accuracy: 0.3432 - val_loss: 5.8881 - val_accuracy: 0.1466
Epoch 5/10
832/832 [==============================] - 688s 827ms/step - loss: 2.4745 - accuracy: 0.5121 - val_loss: 6.7624 - val_accuracy: 0.1529
Epoch 6/10
832/832 [==============================] - 675s 812ms/step - loss: 1.8411 - accuracy: 0.6294 - val_loss: 7.9838 - val_accuracy: 0.1642
Epoch 7/10
832/832 [==============================] - 675s 811ms/step - loss: 1.5191 - accuracy: 0.6889 - val_loss: 9.3100 -

In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = []
label_predictions = []

# Durchlaufe das Testverzeichnis und lade die Bilder
for root, dirs, files in os.walk(test_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            image_names.append(file)
            label_prediction = model.predict(np.expand_dims(image_array, axis=0))
            label_predictions.append(np.argmax(label_prediction))

# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

# Neuer Abschnitt